# Athlete’s Temple Business Chatbot (Gradio + OpenAI/Gemini)

This notebook implements a business chatbot that can:
- **Answer questions** about the business (from `/me/business_summary.txt`),
- **Collect leads** (name, email, notes → CSV),
- **Record feedback / unanswered questions** (tool calls → CSV),
- Run with **OpenAI** *or* **Gemini** (choose in config),
- **Deploy via Gradio** (mobile-friendly).

### 1. Imports

In [6]:
# Core deps
%pip install gradio python-dotenv pandas



  Using cached gradio-5.49.1-py3-none-any.whl.metadata (16 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached audioop_lts-0.2.2-cp313-abi3-macosx_11_0_arm64.whl.metadata (2.0 kB)
  Using cached Brotli-1.1.0-cp313-cp313-macosx_10_13_universal2.whl.metadata (5.5 kB)
  Using cached fastapi-0.119.0-py3-none-any.whl.metadata (28 kB)
  Using cached ffmpy-0.6.3-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.13.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-

In [7]:
%pip -q install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os, time, json, textwrap
import pandas as pd
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

### 2. Configs

In [9]:
# Load .env 
load_dotenv()

# === Choose provider here ===
PROVIDER = os.getenv("PROVIDER", "openai").lower().strip() 

# API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()

# Basic guards
if PROVIDER == "openai" and not OPENAI_API_KEY:
    print("⚠️ Set OPENAI_API_KEY in environment or .env")

# Paths
ME_DIR = Path("me")
SUMMARY_TXT = ME_DIR / "business_summary.txt"
LEADS_CSV = Path("leads.csv")
FEEDBACK_CSV = Path("feedback.csv")

ME_DIR.mkdir(exist_ok=True)

# Create CSVs with headers if absent
if not LEADS_CSV.exists():
    pd.DataFrame(columns=["timestamp","name","email","notes","source"]).to_csv(LEADS_CSV, index=False)
if not FEEDBACK_CSV.exists():
    pd.DataFrame(columns=["timestamp","user_message","model_answer","status","tag"]).to_csv(FEEDBACK_CSV, index=False)

print(f"✅ Provider: {PROVIDER}")
print(f"📄 Summary file: {SUMMARY_TXT.resolve()}")
print(f"🗂️ leads.csv: {LEADS_CSV.resolve()}")
print(f"🗂️ feedback.csv: {FEEDBACK_CSV.resolve()}")


✅ Provider: openai
📄 Summary file: /Users/adamtai/Desktop/GPCS/4. FALL 2025/EECE 798S/Assignments/Assignment 3/me/business_summary.txt
🗂️ leads.csv: /Users/adamtai/Desktop/GPCS/4. FALL 2025/EECE 798S/Assignments/Assignment 3/leads.csv
🗂️ feedback.csv: /Users/adamtai/Desktop/GPCS/4. FALL 2025/EECE 798S/Assignments/Assignment 3/feedback.csv


In [21]:
# --- INIT OPENAI CLIENT SAFELY ---
import os
from openai import OpenAI

# 1) Ensure your API key is set (in .env or environment)
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "OPENAI_API_KEY not set. Add it to your environment or a .env file and rerun this cell."
    )

# 2) Create a single global client the rest of the notebook can use
client = OpenAI()  # picks up OPENAI_API_KEY from the environment

# 3) Optional quick sanity check (lists your accessible models)
try:
    _ = client.models.list()
    print("✅ OpenAI client initialized.")
except Exception as e:
    raise RuntimeError(f"OpenAI init failed: {e}")

✅ OpenAI client initialized.


### 3. Load Business Knowledge
We read `/me/business_summary.txt` as the authoritative context.  

In [23]:
if not SUMMARY_TXT.exists():
    SUMMARY_TXT.write_text(textwrap.dedent("""\
        Athlete's Temple

        Mission: To empower individuals of all levels to unlock their physical and mental potential through expert-led fitness, martial arts, and personalized coaching programs.

        Services Offered:
        • Regular Gym Membership (Full Access to Training Facilities)
        • Martial Arts Classes — Taekwondo, Brazilian Jiu-Jitsu, Muay Thai, and Open Mat Sessions
        • Personal Training Programs with Certified Coaches
        • Calisthenics Classes
        • Snack Bar & Gym Supplements - sports drinks, refreshments, protein bars, and energy bars
        
        Our Team:
        Coach Dan - Founder & Head Coach: A former national Taekwondo athlete and certified strength coach with 15+ years in martial arts instruction and athlete conditioning.
        Coach Park Lee - Head of Personal Training: Certified fitness and wellness specialist, helping clients achieve lasting transformations through tailored programs and lifestyle guidance.
        Coach Tamer - BJJ Lead Instructor: Brazilian Jiu-Jitsu black belt and professional competitor, passionate about technical excellence and personal growth through combat sports.
        Coach Majd - Calisthenics Lead Instructor: A jacked calisthenics athlete with over 8 years of experience, leading multiple classes for different age groups, from kids to adults.
        Coach Lynn - Muay Thai Lead Instructor: Young muay thai prodigy, reigning national champion leading and guiding others on their own journey to similar successes.
        Bassel Hani - Operations & Financial Manager: With 10+ years in fitness operations and finance, Bassel keeps Athlete's Temple running smoothly - managing budgets, memberships, and logistics with precision and efficiency.
        
        Unique Value Proposition:
        Athlete's Temple Training Club stands out as a hybrid fitness and martial arts hub - blending elite coaching, science-backed programming, and a strong community culture. Whether you're training for performance, confidence, or balance, Athlete's Temple provides the structure, motivation, and expert guidance to help you thrive.
    """).strip(), encoding="utf-8")

KB_TEXT = SUMMARY_TXT.read_text(encoding="utf-8")
print("✅ Summary loaded (first 400 chars):\n", KB_TEXT[:400], "...")

✅ Summary loaded (first 400 chars):
 Athlete’s Temple
Mission: To empower individuals of all levels to unlock their physical and mental potential
through expert-led fitness, martial arts, and personalized coaching programs.
Services Offered:
• Regular Gym Membership (Full Access to Training Facilities)
• Martial Arts Classes — Taekwondo, Brazilian Jiu-Jitsu, Muay Thai, and Open Mat Sessions
• Personal Training Programs with Certified ...


### 4. Tools

In [24]:
def record_customer_interest(name: str, email: str, notes: str, source: str="chat"):
    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "name": (name or "").strip(),
        "email": (email or "").strip(),
        "notes": (notes or "").strip(),
        "source": source
    }
    df = pd.read_csv(LEADS_CSV)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(LEADS_CSV, index=False)
    return {"ok": True, "saved": row}

def record_feedback(user_message: str, reason: str="out_of_scope_or_missing"):
    row = {
        "timestamp": datetime.utcnow().isoformat(),
        "user_message": (user_message or "").strip(),
        "model_answer": "",
        "status": "unanswered",
        "tag": reason
    }
    df = pd.read_csv(FEEDBACK_CSV)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(FEEDBACK_CSV, index=False)
    return {"ok": True, "saved": row}

### 5. System prompt & tool schemas

In [ ]:
SYSTEM_DIRECTIVES = textwrap.dedent(f"""
You are the official concierge for Athlete's Temple (hybrid fitness & martial arts club).
Use ONLY the business summary as your ground truth. If a question asks for missing info
(e.g., pricing, schedule, policies), call the tool `record_feedback` with the user question,
give the user a brief friendly message, and invite them to share their contact details by asking
"Would you like to leave your name and email so a coach can follow up with you?"

If the user expresses interest or shares contact details, call the tool `record_customer_interest`
with name, email, and a short note (their message or intent).

Otherwise (when the answer is fully covered by the summary), do NOT ask for contact details
and do NOT mention follow-up.

Be concise, factual, and friendly.
""").strip()

def build_user_content(user_message: str) -> str:
    return textwrap.dedent(f"""
    [BUSINESS SUMMARY]
    {KB_TEXT}

    [USER]
    {user_message}
    """).strip()

# OpenAI tool (function) schemas
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Save a potential customer's contact details and note for follow-up.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name":  {"type": "string", "description": "Customer name, if provided."},
                    "email": {"type": "string", "description": "Customer email, if provided."},
                    "notes": {"type": "string", "description": "Short note about their interest or request."}
                },
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Log a question that couldn't be answered from the business summary.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_message": {"type": "string", "description": "The unanswered question from the user."},
                    "reason": {"type": "string", "enum": ["out_of_scope_or_missing","other"], "description": "Why it was logged."}
                },
                "required": ["user_message"]
            }
        }
    }
]


### 6. Execute model’s tool calls

In [26]:
def _dispatch_tool_call(name: str, arguments: dict):
    try:
        if name == "record_customer_interest":
            return record_customer_interest(
                name=arguments.get("name",""),
                email=arguments.get("email",""),
                notes=arguments.get("notes",""),
                source="chat_auto"
            )
        elif name == "record_feedback":
            return record_feedback(
                user_message=arguments.get("user_message",""),
                reason=arguments.get("reason","out_of_scope_or_missing")
            )
        else:
            return {"ok": False, "error": f"Unknown tool: {name}"}
    except Exception as e:
        return {"ok": False, "error": str(e)}

### 7. Agentic chat with tool loop

In [35]:
def _msg_to_dict(msg):
    """
    Convert OpenAI SDK message object to a plain dict, preserving tool_calls.
    """
    d = {"role": msg.role}
    # content can be None or string
    if getattr(msg, "content", None) is not None:
        d["content"] = msg.content
    else:
        d["content"] = ""

    # Preserve tool_calls if present
    if getattr(msg, "tool_calls", None):
        calls = []
        for tc in msg.tool_calls:
            calls.append({
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments or "{}"
                }
            })
        d["tool_calls"] = calls
    return d


def agentic_chat_once(user_message: str, history_messages: list):
    """
    Single turn with tool-calling:
    1) Add system + user
    2) Call model (allows tools)
    3) If tool_calls present -> run tools, append tool results, call model again
    4) Return final text and updated history
    """
    # 1) Ensure system message at start
    if not history_messages or history_messages[0].get("role") != "system":
        history_messages = [{"role": "system", "content": SYSTEM_DIRECTIVES}] + history_messages

    # Add user with KB context
    history_messages = history_messages + [{"role": "user", "content": build_user_content(user_message)}]

    # 2) First call (tool-choice = auto)
    resp = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        messages=history_messages,
        tools=TOOLS,
        tool_choice="auto",
        temperature=float(os.getenv("OPENAI_TEMPERATURE","0.6")),
        top_p=float(os.getenv("OPENAI_TOP_P","0.9")),
        max_tokens=int(os.getenv("OPENAI_MAX_TOKENS","400"))
    )
    msg = resp.choices[0].message

    # IMPORTANT: append the assistant message WITH ITS tool_calls preserved
    history_messages.append(_msg_to_dict(msg))

    tool_calls = msg.tool_calls or []
    feedback_logged = False

    if tool_calls:
        # 3) Execute each tool call and append a corresponding tool message
        for tc in tool_calls:
            fn_name = tc.function.name
            fn_args = json.loads(tc.function.arguments or "{}")
            result = _dispatch_tool_call(fn_name, fn_args)

            if fn_name == "record_feedback":
                feedback_logged = True

            # Each tool message must reference the EXACT tool_call_id
            history_messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "name": fn_name,
                "content": json.dumps(result)
            })

        # 4) Second call so the model can read tool outputs and craft the final reply
        resp2 = client.chat.completions.create(
            model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
            messages=history_messages,
            temperature=float(os.getenv("OPENAI_TEMPERATURE","0.6")),
            top_p=float(os.getenv("OPENAI_TOP_P","0.9")),
            max_tokens=int(os.getenv("OPENAI_MAX_TOKENS","400"))
        )
        final_msg = resp2.choices[0].message
        text = (final_msg.content or "").strip()

        # Optional: strip the contact nudge unless feedback was actually logged
        if not feedback_logged:
            text = text.replace(
                "Would you like to leave your name and email so a coach can follow up?", ""
            ).strip()

        history_messages.append({"role": "assistant", "content": text})
        return text, history_messages

    # No tools needed → return first response (without the nudge)
    text = (msg.content or "").strip()
    text = text.replace(
        "Would you like to leave your name and email so a coach can follow up?", ""
    ).strip()
    return text, history_messages


### 8. AI & UI bridge

In [36]:
# We’ll keep a parallel "display history" for the Chatbot (tuples).
def respond(user_message, openai_history, display_history):
    openai_history = openai_history or []
    display_history = display_history or []

    try:
        answer, openai_history = agentic_chat_once(user_message, openai_history)
    except Exception as e:
        answer = f"Sorry, there was an issue contacting the model: {e}"

    display_history = display_history + [(user_message, answer)]
    return "", openai_history, display_history

def load_feedback_table():
    try: return pd.read_csv(FEEDBACK_CSV)
    except: return pd.DataFrame(columns=["timestamp","user_message","model_answer","status","tag"])

def load_leads_table():
    try: return pd.read_csv(LEADS_CSV)
    except: return pd.DataFrame(columns=["timestamp","name","email","notes","source"])

### 9. Gradio UI

In [37]:
with gr.Blocks(title="Athlete's Temple — Agentic Bot") as demo:
    gr.Markdown("""
    # Athlete's Temple — Gym Manager Bot 
    • Answers questions from the business summary  
    • Automatically logs feedback or collects leads via tool calls  
    • Mobile-friendly Gradio UI
    """)

    with gr.Tab("Chat"):
        chatbot = gr.Chatbot(height=420, type="tuples", label="Chat")
        with gr.Row():
            msg = gr.Textbox(placeholder="Ask about classes, PT, or our mission…", label="Your Message")
            send = gr.Button("Send", variant="primary")
        clear = gr.Button("Clear Chat")

        # OpenAI-format history + display history
        openai_history_state = gr.State([])   # list of message dicts
        display_history_state = gr.State([])  # list of (user, assistant)

        send.click(
            respond,
            inputs=[msg, openai_history_state, display_history_state],
            outputs=[msg, openai_history_state, chatbot]
        )
        msg.submit(
            respond,
            inputs=[msg, openai_history_state, display_history_state],
            outputs=[msg, openai_history_state, chatbot]
        )
        clear.click(lambda: ([], []), outputs=[openai_history_state, chatbot])

    with gr.Tab("Leads"):
        name = gr.Textbox(label="Name")
        email = gr.Textbox(label="Email")
        notes = gr.Textbox(label="Notes (optional)")
        submit = gr.Button("Submit Lead", variant="primary")
        lead_status = gr.Markdown()

        def _submit_lead(name, email, notes):
            result = record_customer_interest(name or "", email or "", notes or "", source="lead_form")
            return "Thanks! Your info has been recorded." if result.get("ok") else f"Error: {result}"

        submit.click(_submit_lead, inputs=[name,email,notes], outputs=[lead_status])

    with gr.Tab("Logs"):
        gr.Markdown("### Feedback (Unanswered / Info Requests)")
        fb_btn = gr.Button("Refresh Feedback")
        fb_table = gr.Dataframe(headers=["timestamp","user_message","model_answer","status","tag"], wrap=True)

        gr.Markdown("### Leads (Collected)")
        leads_btn = gr.Button("Refresh Leads")
        leads_table = gr.Dataframe(headers=["timestamp","name","email","notes","source"], wrap=True)

        fb_btn.click(load_feedback_table, outputs=[fb_table])
        leads_btn.click(load_leads_table, outputs=[leads_table])

print("✅ UI built. Use demo.launch(...) next.")


✅ UI built. Use demo.launch(...) next.


/var/folders/fr/7v5vvmb91wnbf1t19lh721700000gn/T/ipykernel_74621/2668336450.py:10: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420, type="tuples", label="Chat")


### 10. Launch

In [ ]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://2ec0afd7654bd295e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/var/folders/fr/7v5vvmb91wnbf1t19lh721700000gn/T/ipykernel_74621/1310164506.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
/var/folders/fr/7v5vvmb91wnbf1t19lh721700000gn/T/ipykernel_74621/1310164506.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
